In [1]:
# Import libraries
import pandas as pd

In [2]:
main = pd.read_csv("../data/interim/bangkok_traffy_cleaned.csv")
external = pd.read_csv("../data/external/bangkok_external_data.csv")


In [ ]:
# Merge main dataset with external daily features
# Build comparable 'date' columns (prefer timestamp; robust parsing; fallback to year/month/day)
if 'timestamp' in main.columns:
    # Robust vectorized parsing: try fast parse first, coercing errors to NaT
    main['timestamp_parsed'] = pd.to_datetime(main['timestamp'].astype(str), errors='coerce', utc=True, infer_datetime_format=True)
    # If there are remaining NaT values, try a more flexible parse for those rows
    if main['timestamp_parsed'].isna().any():
        try:
            mask = main['timestamp_parsed'].isna()
            main.loc[mask, 'timestamp_parsed'] = pd.to_datetime(main.loc[mask, 'timestamp'].astype(str), errors='coerce', utc=True, format='mixed')
        except Exception:
            pass

    # Convert to local date (Asia/Bangkok) when tz-aware, otherwise use the UTC date
    try:
        main['date'] = main['timestamp_parsed'].dt.tz_convert('Asia/Bangkok').dt.date
    except Exception:
        main['date'] = main['timestamp_parsed'].dt.date

    # If parsing still failed for some rows, try building date from year/month/day if available
    if main['date'].isna().any() and set(['year','month','day']).issubset(main.columns):
        mask2 = main['date'].isna()
        try:
            main.loc[mask2, 'date'] = pd.to_datetime(main.loc[mask2, ['year','month','day']]).dt.date
        except Exception:
            main.loc[mask2, 'date'] = pd.to_datetime(main.loc[mask2].apply(lambda r: f"{int(r['year'])}-{int(r['month']):02d}-{int(r['day']):02d}", axis=1), errors='coerce').dt.date

    # Final check: if still no dates, raise informative error
    if main['date'].isna().all():
        raise ValueError('Failed to parse any timestamps into dates. Check timestamp formats.')

    # drop helper column
    if 'timestamp_parsed' in main.columns:
        main.drop(columns=['timestamp_parsed'], inplace=True)
elif set(['year','month','day']).issubset(main.columns):
    main['date'] = pd.to_datetime(main[['year','month','day']]).dt.date
else:
    raise ValueError('Main dataset has no timestamp or year/month/day columns to build date')

# Normalize external date to plain date objects
external['date'] = pd.to_datetime(external['date']).dt.date

# Merge on date (left join to keep all main rows)
merged = main.merge(external, on='date', how='left', suffixes=(None, '_ext'))

# Basic merge diagnostics
pm25_col = 'pm25_avg' if 'pm25_avg' in merged.columns else None
rows_with_external = 0
if pm25_col:
    rows_with_external = merged[pm25_col].notna().sum()

print(f'Merged rows: {len(merged):,}')
if pm25_col:
    print(f'Rows with PM2.5 data: {rows_with_external:,} ({rows_with_external/len(merged)*100:.1f}%)')
else:
    print('External PM2.5 column not found in external dataset')

C:\Users\kkeaa\AppData\Local\Temp\ipykernel_28260\3787185221.py:5: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  main['timestamp_parsed'] = pd.to_datetime(main['timestamp'].astype(str), errors='coerce', utc=True, infer_datetime_format=True)


Merged rows: 403,830
Rows with PM2.5 data: 403,818 (100.0%)
✓ Saved merged dataset to: ../data/processed/traffy_with_external.csv
✓ Saved merged dataset to: ../data/processed/traffy_with_external.csv


,ticket_id,type,organization,comment,photo,photo_after,coords,address,subdistrict,district,...,month_ext,day_of_week_ext,quarter,day_of_year,is_weekend,is_holiday,holiday,holiday_type,weather_severity,year_month
0,2021-FYJTFP,{ความสะอาด},เขตบางซื่อ,ขยะเยอะ,https://storage.googleapis.com/traffy_public_b...,NaN,"100.53084,13.81865",12/14 ถนน กรุงเทพ- นนทบุรี แขวง บางซื่อ เขตบาง...,บางซื่อ,บางซื่อ,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2021-CGPMUN,"{น้ำท่วม,ร้องเรียน}","เขตประเวศ,ฝ่ายโยธา เขตประเวศ",น้ำท่วมเวลาฝนตกและทะลุเข้าบ้านเดือดร้อนมากทุกๆ...,https://storage.googleapis.com/traffy_public_b...,https://storage.googleapis.com/traffy_public_b...,"100.66709,13.67891",189 เฉลิมพระเกียรติ ร.9 แขวง หนองบอน เขต ประเว...,หนองบอน,ประเวศ,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2021-7XATFA,{สะพาน},เขตสาทร,สะพานลอยปรับปรุงไม่เสร็จตามกำหนด\nปากซอย สาทร12,https://storage.googleapis.com/traffy_public_b...,NaN,"100.52649,13.72060",191/1 ถนน สาทรเหนือ แขวง สีลม เขตบางรัก กรุงเท...,ยานนาวา,สาทร,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2021-9U2NJT,{น้ำท่วม},"เขตบางซื่อ,ฝ่ายโยธา เขตบางซื่อ",น้ำท่วม,https://storage.googleapis.com/traffy_public_b...,https://storage.googleapis.com/traffy_public_b...,"100.53099,13.81853",12/14 ถนน กรุงเทพ- นนทบุรี แขวง บางซื่อ เขตบาง...,บางซื่อ,บางซื่อ,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2021-DVEWYM,"{น้ำท่วม,ถนน}","เขตลาดพร้าว,ฝ่ายโยธา เขตลาดพร้าว",ซอยลาดพร้าววังหิน 75 ถนนลาดพร้าววังหิน แขวงลาด...,https://storage.googleapis.com/traffy_public_b...,NaN,"100.59165,13.82280",702 ถ. ลาดพร้าววังหิน แขวงลาดพร้าว เขตลาดพร้าว...,ลาดพร้าว,ลาดพร้าว,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


min year: 2021
max year: 2023


In [ ]:
# Save merged dataset
import os
os.makedirs('../data/processed', exist_ok=True)
out_path = '../data/processed/traffy_with_external.csv'
merged.to_csv(out_path, index=False)
print(f'✓ Saved merged dataset to: {out_path}')

# Show sample and year range from merged (if year exists)
from IPython.display import display
display(merged.head())
if 'year' in merged.columns:
    print('min year:', int(merged['year'].min()))
    print('max year:', int(merged['year'].max()))
else:
    print('No `year` column found in merged dataset')